回顾一下管线。

典型 RLHF/ RLAIF 管线：SFT → 训练奖励模型（RM，多为成对偏好 / Bradley–Terry）→ 用 PPO / GRPO 在 RM（+KL 正则）上做策略优化；近年也常把 PRM（过程奖励）混用，给中间推理步骤提供更细粒度信号。PPO 来自经典 RL（actor–critic），GRPO 则用“组内平均得分”作基线，省去价值网络、节省显存并稳定方差（DeepSeek 系列把它用热了）。 ￼ ￼

**奖励模型（RM）设计里的典型问题与成因**

- 过优化 / “Goodhart 化”与分布外（OOD）外推：当策略反复“对着” RM 优化时，会出现“代理指标被过度优化而偏离人类真偏好”的现象；在合成设定和真实任务上都被系统地观察到（Goodhart’s law）。迭代式 RLHF 会放大这一动态。根因常是 RM 在策略分布移位后对 OOD 文本打分自信但错误；缓解：更强 KL/自适应 KL、早停、重训 RM（迭代 RLHF）、不确定性/集成 RM、行为支持/过滤 OOD 的优化方法等；
- 数据与标注噪声：偏好数据存在歧义 / 错误标注会显著影响 RM 的稳健性；对“偏好强度”建模、投票式多 RM 判别、鲁棒学习（把离群标注当稀疏噪声）等能缓解；
- 文本长度偏置（length bias / length hacking）：RM 常“偏爱更长的回答”，导致策略学会“写长文拿高分”，而非真正质量提升；PPO 阶段的奖励与 KL / 长度的相互作用，会进一步推长输出；缓解：奖励归一化（按长度）、长度正则 / 约束、分段 / 逐 token 奖励、PRM / 分段 RM；
- 复合奖励的相关性与失真：把多个子 RM 线性/加权组合时，分量相关会带来新的过优化通道；需要去相关或保守 / 悲观加权；
- 方差过低：信号“糊成一片”，策略难分辨好坏；方差适中能加快学习；近期工作甚至通过**有原则地“拉高奖励方差”**来加速训练（保持期望与偏好排序不变的前提）；策略侧可做优势白化、分 prompt 归一化、组内基线 / 价值基线等；

**强化阶段（PPO / GRPO）里的主要问题与深层原因**

- KL 正则与分布漂移：KL 过小 → 过优化 + OOD 文本；KL 过大 → 学不到新行为；自适应 KL / 参考策略重置能改善稳定性；
- 方差与基线估计：长序列、终端稀疏奖励导致策略梯度方差随“地平线”增大；PPO 依赖价值网络减小方差，GRPO 用“组内均值基线”在不引入 critic 的情况下降低方差与显存占用（组太小或样本同质化会出现“近零方差 / 学习停滞”）；
- 信号粒度过粗 → 信用分配困难：只在打分会把同一优势复制给整段 token，难以把“功劳 / 过错”定位到关键步骤，易放大长度偏置与“空想 / 过度思考”；PRM、token / segment 奖励与宏动作（macro actions）能在机制上缓解；
- 迭代 RLHF 的动态：多轮“重训 RM → 再优化策略”能减轻一次性过优化，但也可能在循环里累积偏差，需要设计数据转移、初始化和奖励函数选择的策略；

**rollout（推理长度）带来的问题与成因**
- 梯度方差 / 信用分配随长度放大：长地平线、稀疏末端奖励 → 高方差、学习慢；错误会沿链路累积；
- 奖励与 KL 的长度耦合：奖励常按全序列给分，KL 又是逐 token 聚合，二者耦合容易诱发“越写越长 / 或过度保守”；
- 计算与采样压力：长 rollout 使采样成本、显存占用和优化噪声齐升；实践中常用“逐步加长 / 早停”“长度配额 / 惩罚”与“参考策略重置”等工程技巧；